## SVM with feature extraction using neural nets

In [ ]:
import torch
from torchvision import datasets, transforms, models
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, Subset
import numpy as np
import random

def extract_features(data_dir, sample_size=100):
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
    
    # Load dataset and sample a subset
    full_dataset = datasets.ImageFolder(data_dir, transform=transform)
    indices = list(range(len(full_dataset)))
    sampled_indices = random.sample(indices, sample_size)
    dataset = Subset(full_dataset, sampled_indices)
    loader = DataLoader(dataset, batch_size=len(dataset), shuffle=False)
    
    # Use ResNet50 for feature extraction
    model = models.resnet50(pretrained=True)
    model = torch.nn.Sequential(*(list(model.children())[:-1]))  # Remove last layer
    model.eval()

    features = []
    labels = []
    
    with torch.no_grad():
        for inputs, label_batch in loader:
            # Extract features
            output = model(inputs).flatten(start_dim=1)
            features.append(output)
            labels.append(label_batch)
    
    features = torch.cat(features).numpy()
    labels = torch.cat(labels).numpy()
    
    return features, labels

def train_svm(features, labels):
    # Split the data into training and validation sets
    X_train, X_val, y_train, y_val = train_test_split(features, labels, test_size=0.2, random_state=42)
    
    # Train SVM
    clf = SVC(kernel='linear', C=1)
    clf.fit(X_train, y_train)
    
    # Validate
    predictions = clf.predict(X_val)
    accuracy = accuracy_score(y_val, predictions)
    print(f"Validation Accuracy: {accuracy:.4f}")

# Load data and preprocess
data_dir = '/Users/mrinoyb2/git/Wound-Detection/Wound_dataset copy' 
features, labels = extract_features(data_dir, sample_size=100)  # Adjust sample_size as needed

# Train and evaluate the SVM model
train_svm(features, labels)